In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import scipy
from sympy.matrices import Matrix, eye
#
from SBMLLint.games.message import Message

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
simple = load_file_from_curated_data(40)
for r in simple.reactions:
  #if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

Reaction1: Br + BrO3 -> HBrO2 + HOBr
Reaction2: Br + HBrO2 -> 2.00 HOBr
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr
Reaction5: Ce -> Br


In [4]:
m = Message(simple)
m

{BrO3};{Ce};{HBrO2};{HOBr};{Br}

In [6]:
m.stoichiometry_matrix

,Reaction1,Reaction2,Reaction3,Reaction4,Reaction5
BrO3,-1.0,0.0,-1.0,1.0,0.0
Ce,0.0,0.0,1.0,0.0,-1.0
HBrO2,1.0,-1.0,1.0,-2.0,0.0
HOBr,1.0,2.0,0.0,1.0,0.0
Br,-1.0,-1.0,0.0,0.0,1.0


In [42]:
m.reactions

[Reaction1: Br + BrO3 -> HBrO2 + HOBr; Br * BrO3 * k1 * BZ,
 Reaction2: Br + HBrO2 -> 2.00 HOBr; Br * HBrO2 * k2 * BZ,
 Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2; BrO3 * HBrO2 * k3 * BZ,
 Reaction4: 2.00 HBrO2 -> BrO3 + HOBr; pow(HBrO2, 2) * k4 * BZ,
 Reaction5: Ce -> Br; Ce * k5 * BZ]

In [43]:
m.molecules

['Ce', 'HBrO2', 'HOBr', 'Br', 'BrO3']

In [44]:
m.stoichiometry_matrix

,Reaction1,Reaction2,Reaction3,Reaction4,Reaction5
Ce,0.0,0.0,1.0,0.0,-1.0
HBrO2,1.0,-1.0,1.0,-2.0,0.0
HOBr,1.0,2.0,0.0,1.0,0.0
Br,-1.0,-1.0,0.0,0.0,1.0
BrO3,-1.0,0.0,-1.0,1.0,0.0


In [5]:
simple.getMolecule('Br')

Br

In [45]:
m

In [46]:
mat_df = m.stoichiometry_matrix
m.decomposeMatrix(mat_df).T


# mat_t = mat_df.T
# idx_mat_t = mat_t.index
# # LU decomposition
# sym_m = Matrix(mat_df.T)
# lower, upper, perm = sym_m.LUdecomposition()
# permuted_m = sym_m.permuteFwd(perm)
# new_idx_mat_t = list(Matrix(idx_mat_t).permuteFwd(perm))
# new_idx_mat_t
# new_df = pd.DataFrame(np.array(permuted_m).astype(np.float64), 
#                       index=new_idx_mat_t, 
#                       columns=mat_t.columns).T
# new_df

,Ce,HBrO2,HOBr,Br,BrO3
Reaction3,1.0,1.0,0.0,0.000000,-1.000000
Reaction2,0.0,-1.0,2.0,-1.000000,0.000000
Reaction1,0.0,0.0,3.0,-2.000000,-1.000000
Reaction5,0.0,0.0,0.0,1.333333,-0.333333
Reaction4,0.0,0.0,0.0,0.000000,0.000000


In [54]:
print(perm)
m.stoichiometry_matrix.T

[[0, 2], [3, 4]]


,Ce,HBrO2,HOBr,Br,BrO3
Reaction1,0.0,1.0,1.0,-1.0,-1.0
Reaction2,0.0,-1.0,2.0,-1.0,0.0
Reaction3,1.0,1.0,0.0,0.0,-1.0
Reaction4,0.0,-2.0,1.0,0.0,1.0
Reaction5,-1.0,0.0,0.0,1.0,0.0


In [53]:
m.permuted_matrix.T

,Ce,HBrO2,HOBr,Br,BrO3
Reaction3,1.0,1.0,0.0,0.0,-1.0
Reaction2,0.0,-1.0,2.0,-1.0,0.0
Reaction1,0.0,1.0,1.0,-1.0,-1.0
Reaction5,-1.0,0.0,0.0,1.0,0.0
Reaction4,0.0,-2.0,1.0,0.0,1.0


In [48]:
m.lower_inverse

,Reaction3,Reaction2,Reaction1,Reaction5,Reaction4
0,1.0,0.000000,0.000000,0.0,0.0
1,0.0,1.000000,0.000000,0.0,0.0
2,0.0,1.000000,1.000000,0.0,0.0
3,1.0,0.333333,-0.666667,1.0,0.0
4,0.0,-1.000000,1.000000,0.0,1.0


In [50]:
perm

[[0, 2], [3, 4]]

In [32]:
# mat_df = m.stoichiometry_matrix
# mat_t = mat_df.T

# mat_t
# idx_mat_t = mat_t.index
# # # LU decomposition
# mat_lu = scipy.linalg.lu(mat_t)
# # # inverse pivot matrix
# p_inv = scipy.linalg.inv(mat_lu[0])
# pivot_index = [list(k).index(1) for k in p_inv]
# new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
# # # row reduced matrix
# row_reduced = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
# row_reduced

In [34]:
m = Matrix(mat_df.T)
L, U, perm= m.LUdecomposition()
L.inv()

Matrix([
[  1,                 0,                  0,   0,   0],
[  0,               1.0,                  0,   0,   0],
[  0,               1.0,                1.0,   0,   0],
[1.0, 0.333333333333333, -0.666666666666667, 1.0,   0],
[  0,              -1.0,                1.0,   0, 1.0]])

In [35]:
U

Matrix([
[1.0,  1.0, 0.0,              0.0,               -1.0],
[  0, -1.0, 2.0,             -1.0,                  0],
[  0,    0, 3.0,             -2.0,               -1.0],
[  0,    0,   0, 1.33333333333333, -0.333333333333333],
[  0,    0,   0,                0,                  0]])

In [36]:
permuted_m = m.permuteFwd(perm)
permuted_m

Matrix([
[ 1.0,  1.0, 0.0,  0.0, -1.0],
[ 0.0, -1.0, 2.0, -1.0,  0.0],
[ 0.0,  1.0, 1.0, -1.0, -1.0],
[-1.0,  0.0, 0.0,  1.0,  0.0],
[ 0.0, -2.0, 1.0,  0.0,  1.0]])

In [37]:
list(Matrix(['a', 'b', 'c', 'd']).permuteFwd(perm))

IndexError: Index out of range: a[4]